In [37]:
import requests
import urllib.parse
import json

In [22]:
import bs4
from collections import defaultdict
from pprint import pprint

In [32]:
base_url = 'https://www.supremecourt.ohio.gov/JudSystem/trialCourts/'

In [33]:
resp = requests.get(base_url)

In [5]:
soup = bs4.BeautifulSoup(resp.text)

/Volumes/exlinux/virt2/ve/jup/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [6]:
court_table = soup.select("div#mainContent2 table")[0]

In [40]:
data = {}
county_name = ''
for tr in court_table.select('tbody tr'):
    tds = tr.select('td')
    if len(tds) == 3:
        (county_td, court_td, local_rules_td) = tds
        raw_county_name = county_td.text.strip()
        if raw_county_name:
            county_name = raw_county_name
            data[county_name] = {}
        if court_td.a:
            court_name = court_td.a.text
            url = court_td.a.attrs['href']
            url = urllib.parse.urljoin(base_url, url)
            data[county_name][court_name] = {'url': url}
        else:
            court_name = court_td.text
            data[county_name][court_name] = {}
        if local_rules_td.a:
            data[county_name][court_name]['local_rules'] = urllib.parse.urljoin(base_url, 
                                                                                local_rules_td.a.attrs['href'])
 
pprint(data)
        


{'Adams': {'Adams County Court': {'local_rules': 'https://www.supremecourt.ohio.gov/Clerk/Local '
                                                 'Rules/adamsCC.pdf',
                                  'url': 'https://adamscountyoh.gov/County-Court-Judge.asp'},
           'Adams County Court of Common Pleas': {'local_rules': 'https://www.adamscountyoh.gov/files/Adams_County_OH_COURT-RULES_1-4-2011.pdf',
                                                  'url': 'https://adamscountyoh.gov/Common-Pleas-Judge.asp'}},
 'Allen': {'Allen County Court of Common Pleas,  Juvenile Division': {'local_rules': 'http://www.allenohiojuvenile.com/downloads/LOCAL '
                                                                                     'RULES.pdf',
                                                                      'url': 'http://www.allenohiojuvenile.com/'},
           'Allen County Court of Common Pleas, Domestic Relations Division': {'local_rules': 'http://www.allencountyohio.com/wp-con

In [41]:
with open('../court-links.js', 'w') as outfile:
    json.dump(data, outfile, indent=2)